In [1]:
# Imports
import requests
import json
from pprint import pprint
from config import api_key

In [2]:
# Imports for calculations
import pandas as pd
import numpy as np

In [3]:
# Base Url for 
url = "https://api.usa.gov/crime/fbi/cde/"


In [4]:
# Agency Query Url
query = "byStateAbbr"
value = "NC"
agency_query_url = url + "agency/" + query + "/" + value + "?API_KEY=" + api_key

print(agency_query_url)

https://api.usa.gov/crime/fbi/cde/agency/byStateAbbr/NC?API_KEY=VZapwhFihrweB1CK2Aorx06jomfZ97WTTsx2lpjX


In [5]:
# Get AGENCY data
agency_response = requests.get(agency_query_url)
agency_response_json = agency_response.json()

# Get the JSON format
#pprint(agency_response_json)

In [6]:
ori_list = [agency['ori'] for agency in agency_response_json]
#print("List of ORIs:", ori_list)

In [7]:
offense = "violent-crime"
start_year = "2013"
end_year = "2023"
crime_responses = []
for ori in ori_list:
    target_url = url + f"summarized/agency/{ori}/{offense}?from={start_year}&to={end_year}&API_KEY={api_key}"

    #print(target_url)
    responses = requests.get(target_url)
    responses_json = responses.json()
    crime_responses.append(responses_json)
# Get the JSON format


In [8]:
crime_df = pd.DataFrame()
for responses_json in crime_responses:
    #crime_data = responses_json.get('output', [])
    crime_data_df = pd.DataFrame(responses_json)
    crime_df = pd.concat([crime_df, crime_data_df], ignore_index=True)
print(crime_df.head())


         ori  data_year        offense  cleared  actual
0  NC0010000       2013  violent-crime       84     109
1  NC0010000       2014  violent-crime       81     123
2  NC0010000       2015  violent-crime       97     114
3  NC0010000       2016  violent-crime       77      95
4  NC0010000       2017  violent-crime       34      46


In [9]:
#Creating Data frames
agency_df = pd.DataFrame(agency_response_json)
#crime_df = pd.DataFrame(crime_response_json['output'])

#Merging the 2 dataframes together
merge_df = pd.merge(agency_df, crime_df, on='ori', how='inner')
print(merge_df.head())
merge_df.to_csv("Output/merged_data.csv", index_label="")

         ori                       agency_name  agency_id      state_name  \
0  NC0010000  Alamance County Sheriff's Office      10873  North Carolina   
1  NC0010000  Alamance County Sheriff's Office      10873  North Carolina   
2  NC0010000  Alamance County Sheriff's Office      10873  North Carolina   
3  NC0010000  Alamance County Sheriff's Office      10873  North Carolina   
4  NC0010000  Alamance County Sheriff's Office      10873  North Carolina   

  state_abbr   division_name region_name region_desc county_name  \
0         NC  South Atlantic       South  Region III    ALAMANCE   
1         NC  South Atlantic       South  Region III    ALAMANCE   
2         NC  South Atlantic       South  Region III    ALAMANCE   
3         NC  South Atlantic       South  Region III    ALAMANCE   
4         NC  South Atlantic       South  Region III    ALAMANCE   

  agency_type_name  nibrs          nibrs_start_date   latitude   longitude  \
0           County   True  2019-01-01T00:00:00.000